In [5]:
import pandas as pd

In [ ]:
amazon_df = pd.read_parquet("amazon_reviews_with_embeddings.parquet")

In [ ]:
print(f"Shape of Amazon Reviews Data: {amazon_df.shape}")
print(f"Columns of Amazon Reviews Data: {amazon_df.columns}")

## Drop Null Values

In [ ]:
amazon_df.dropna(subset=['summary','reviewText', 'reviewerName'],inplace=True)

In [ ]:
amazon_df.drop(columns=['image', 'vote', 'style', 'reviewTime'],inplace=True)

In [ ]:
amazon_df.shape

After dropping null review Text, we will see how embeddings are:

In [ ]:
amazon_df['embedding'].isna().sum()

There are still 156 rows with null embeddings:

In [ ]:
print(amazon_df.loc[amazon_df['embedding'].isna(), 'reviewText'])

In [ ]:
print(amazon_df.loc[32620,'reviewText'])

This is Video reivew, so we will drop these rows. 

In [ ]:
amazon_df.dropna(subset=['embedding'], inplace=True)

## Analyze One Review Products

#### Print Some Reviews Belong to One Review Products

In [ ]:
review_counts = amazon_df.groupby('asin')['reviewText'].count()
one_review_asins = review_counts[review_counts == 1].index
one_review_df = amazon_df[amazon_df['asin'].isin(one_review_asins)].copy()

In [ ]:
for i, review in enumerate(one_review_df['reviewText'].head(20), 1):
    print(f"--- Review {i} ---\n{review}\n")

Most of these reviews has a lot of information, Let's print one word one review products. 

In [ ]:
one_review_df['word_count'] = amazon_df_one_review['reviewText'].str.split().apply(len)

amazon_df_sorted = one_review_df.sort_values(by='word_count')

amazon_df_sorted = amazon_df_sorted.reset_index(drop=True)

for i, row in amazon_df_sorted.head(10).iterrows():
    print(f"--- Review {i+1} ---")
    print(f"Word Count: {row['word_count']}")
    print(f"{row['reviewText']}\n")

Since we use sentence embeddings, one-word reviews from single-review products provide little meaningful information and instead introduce noise. Therefore, we will eliminate such products from our dataset.

In [ ]:
one_word_review_asins = one_review_df[one_review_df['word_count'] <= 1]['asin']

amazon_df_cleaned = amazon_df[~amazon_df['asin'].isin(one_word_review_asins)]

In [ ]:
amazon_df_cleaned.shape

In [ ]:
unique_asins = amazon_df_cleaned['asin'].unique()

In [ ]:
# upload labels
amazon_df_labels = pd.read_pickle("Data/amazon_df_labels.pkl")

In [ ]:
# Merge amazon_df_labels with amazon_df_cleaned on 'asin'
asin_labels_clean_review_df = amazon_df_labels[amazon_df_labels['asin'].isin(amazon_df_cleaned['asin'].unique())]

In [ ]:
asin_labels_clean_review_df = asin_labels_clean_review_df.drop_duplicates(subset='asin')

In [ ]:
print(f"After dropping, the size of the data is: {len(ains_labels_clean_review_df)}")

In [ ]:
asin_labels_clean_review_df['match'].value_counts()

In [ ]:
asin_labels_clean_review_df.shape

In [ ]:
# Save cleaned asins and labels:
asin_labels_clean_review_df.to_csv("Data/asin_labels_clean_review_df.csv", index=False)